In [45]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from math import sqrt
import catboost
import xgboost

In [37]:
def wrangle(path):
    df = pd.read_csv(path, parse_dates=['dt'])
    df['Year'] = pd.to_datetime(df['dt']).dt.year
    df['Month'] = pd.to_datetime(df['dt']).dt.month
    df.set_index('dt', inplace=True)
    df.dropna(inplace=True)

    cols = ['LandAndOceanAverageTemperature', 'LandAndOceanAverageTemperatureUncertainty', 'LandMaxTemperatureUncertainty', 'LandMinTemperatureUncertainty','LandAverageTemperatureUncertainty']
    df.drop(columns=cols, inplace=True) 


    df['lag_1'] = df['LandAverageTemperature'].shift(1)
    df['lag_2'] = df['LandAverageTemperature'].shift(2)
    df['lag_3'] = df['LandAverageTemperature'].shift(3)
    df.dropna(inplace=True)
    return df

In [38]:
df = wrangle('data/GlobalTemperatures.csv')

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1925 entries, 1850-04-01 to 2010-08-01
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   LandMaxTemperature      1925 non-null   float64
 1   LandMinTemperature      1925 non-null   float64
 2   LandAverageTemperature  1925 non-null   float64
 3   Year                    1925 non-null   int32  
 4   Month                   1925 non-null   int32  
 5   lag_1                   1925 non-null   float64
 6   lag_2                   1925 non-null   float64
 7   lag_3                   1925 non-null   float64
dtypes: float64(6), int32(2)
memory usage: 120.3 KB


In [40]:
X = df.drop(columns='LandAverageTemperature')
y = df['LandAverageTemperature']

In [41]:
model = RandomForestRegressor()
model.fit(X, y)

RandomForestRegressor()

In [42]:
feat_imp = pd.Series(model.feature_importances_, index=X.columns)
print(feat_imp.sort_values(ascending=False))

LandMinTemperature    0.862454
LandMaxTemperature    0.133662
Year                  0.001423
lag_3                 0.000785
lag_1                 0.000712
lag_2                 0.000652
Month                 0.000313
dtype: float64


In [43]:
df.drop(columns="LandMaxTemperature", inplace=True)

In [ ]:
tscv = TimeSeriesSplit(n_splits=10)
rmse_scores = []

for train_index, test_index in tscv.split(X):
    X_train, X_test